In [17]:
import numpy as np


def analyze_npy_file(file_path):
    try:
        # 读取 .npy 文件
        data = np.load(file_path)
        # 找到最大值的索引
        max_index = np.unravel_index(np.argmax(data), data.shape)
        # 获取数组的大小
        array_size = data.shape

        index = [0,0]
        index[0] = -20 + 40 * float(max_index[0]) / float(array_size[0]-1)
        index[1] = -20 + 40 * float(max_index[1]) / float(array_size[0]-1)

        # print(f"最大值所在的位置: {index}")
        # print(f"最大值所在的位置: {max_index}")
        # print(f"数组的大小: {array_size}")
        return(index[0],index[1])
    except FileNotFoundError:
        print("错误: 文件未找到!")
    except Exception as e:
        print(f"错误: 发生了一个未知错误: {e}")


if __name__ == "__main__":
    file_path = 'fk_body_wave\\20240825000000_hour_12_fk.npy'  # 请替换为实际的 .npy 文件路径
    analyze_npy_file(file_path)
    

In [18]:
import numpy as np

def calculate_azimuth_and_total_slow(s_x, s_y):

    # 计算方位角
    azimuth = np.arctan2(s_y, s_x)
    # 计算总的慢度
    total_slow = np.sqrt(s_x ** 2 + s_y ** 2)
    return azimuth, total_slow

# 示例慢度值
file_path = 'fk_body_wave\\20240825000000_hour_12_fk.npy'  # 请替换为实际的 .npy 文件路径

s_x ,s_y = analyze_npy_file(file_path)

# 调用函数进行计算
azimuth, total_slow = calculate_azimuth_and_total_slow(s_x, s_y)

print(f"方位角（弧度）: {azimuth}")
print(f"方位角（度）: {np.rad2deg(azimuth)}")
print(f"总的慢度: {total_slow}")

方位角（弧度）: -2.408777551803287
方位角（度）: -138.01278750418336
总的慢度: 3.3634060117684275


In [22]:
from geopy.distance import geodesic
import numpy as np

def calculate_search_range(
    station_lat, station_lon,
    observed_azimuth, delta_azimuth,
    min_distance_km, max_distance_km,
    num_angle_samples=10
):
    """
    根据方位角参数自动计算经纬度搜索范围
    :param delta_azimuth: 方位角误差范围（度）
    :param min_distance_km: 最小震中距（公里）
    :param max_distance_km: 最大震中距（公里）
    :param num_angle_samples: 方位角采样点数
    :return: (min_lat, max_lat, min_lon, max_lon)
    """
    # 生成方位角采样点
    angles = np.linspace(observed_azimuth - delta_azimuth,
                        observed_azimuth + delta_azimuth,
                        num_angle_samples)
    angles = np.mod(angles, 360)  # 规范化到0-360度
    
    # 生成距离采样点
    distances = np.linspace(min_distance_km, max_distance_km, 5)
    
    # 收集所有边界点
    boundary_points = []
    for angle in angles:
        for dist in distances:
            # 计算地理坐标
            p = geodesic(kilometers=dist).destination(
                point=(station_lat, station_lon),
                bearing=angle
            )
            boundary_points.append((p.latitude, p.longitude))
    
    # 计算经纬度范围（扩展1%防止边缘截断）
    lats = [p[0] for p in boundary_points]
    lons = [p[1] for p in boundary_points]
    
    lat_span = max(lats) - min(lats)
    lon_span = max(lons) - min(lons)
    
    return (
        min(lats) - 0.01 * lat_span,
        max(lats) + 0.01 * lat_span,
        min(lons) - 0.01 * lon_span,
        max(lons) + 0.01 * lon_span
    )

# 示例调用
if __name__ == "__main__":
    # 台站参数
    station_lat = 34.0522
    station_lon = -118.2437
    
    # 观测参数
    theta = 120.0      # 中心方位角
    delta_theta = 0 # 方位角误差范围
    min_dist = 50      # 最小震中距（公里）
    max_dist = 300     # 最大震中距（公里）
    
    # 自动计算搜索范围
    min_lat, max_lat, min_lon, max_lon = calculate_search_range(
        station_lat, station_lon,
        observed_azimuth=theta,
        delta_azimuth=delta_theta,
        min_distance_km=min_dist,
        max_distance_km=max_dist
    )
    
    # 输出结果
    print(f"纬度范围：{min_lat:.3f}° ~ {max_lat:.3f}°")
    print(f"经度范围：{min_lon:.3f}° ~ {max_lon:.3f}°")

纬度范围：32.657° ~ 33.837°
经度范围：-117.799° ~ -115.451°


In [ ]:
import numpy as np
from obspy.taup import TauPyModel
from geopy.point import Point
from geographiclib.geodesic import Geodesic


def find_source_location(
    station_lat, station_lon,  # 台站经纬度
    min_lat, max_lat, d_lat,  # 网格纬度范围及步长（度）
    min_lon, max_lon, d_lon,  # 网格经度范围及步长（度）
    min_depth, max_depth, d_depth,  # 深度范围及步长（公里）
    observed_slowness, observed_azimuth,  # 观测慢度（秒/公里）和方位角（度）
    model_name="iasp91"  # 速度模型（如：iasp91, prem）
):
    # 初始化参数
    model = TauPyModel(model=model_name)
    min_error = float('inf')
    best_location = None

    # 生成网格点
    lats = np.arange(min_lat, max_lat + d_lat, d_lat)
    lons = np.arange(min_lon, max_lon + d_lon, d_lon)
    depths = np.arange(min_depth, max_depth + d_depth, d_depth)

    # 遍历所有网格点
    for lat in lats:
        for lon in lons:
            for depth in depths:
                try:
                    # 计算震源与台站的距离和方位角
                    src_point = Point(lat, lon)
                    sta_point = Point(station_lat, station_lon)

                    # 计算台站到震源的距离和方位角
                    result = Geodesic.WGS84.Inverse(sta_point.latitude, sta_point.longitude, src_point.latitude, src_point.longitude)
                    distance_km = result['s12'] / 1000  # 距离，单位转换为公里
                    r = distance_km
                    azimuth = result['azi1']
                    azimuth = (azimuth + 360) % 360  # 将方位角转换到 0 - 360 度范围

                    # 计算理论P波走时（需转换震中距为度）
                    distance_deg = distance_km / 111.195  # 1度≈111.195公里
                    arrivals = model.get_travel_times(
                        source_depth_in_km=depth,
                        distance_in_degree=distance_deg,
                        phase_list=["P"]
                    )
                    if not arrivals:
                        continue  # 无射线路径，跳过

                    # 提取走时
                    t = arrivals[0].time  # 原始震中距的走时

                    # 数值微分计算理论慢度（dt/dr）
                    delta_r = 1.0  # 小增量（公里）
                    # 计算r+delta_r的走时
                    new_src_point = Geodesic.WGS84.Direct(sta_point.latitude, sta_point.longitude, azimuth, (distance_km + delta_r) * 1000)
                    new_distance_plus_deg = (distance_km + delta_r) / 111.195
                    arrivals_plus = model.get_travel_times(
                        source_depth_in_km=depth,
                        distance_in_degree=new_distance_plus_deg,
                        phase_list=["P"]
                    )
                    if not arrivals_plus:
                        continue
                    t_plus = arrivals_plus[0].time

                    # 计算r-delta_r的走时
                    if distance_km - delta_r > 0:
                        new_src_point = Geodesic.WGS84.Direct(sta_point.latitude, sta_point.longitude, azimuth, (distance_km - delta_r) * 1000)
                        new_distance_minus_deg = (distance_km - delta_r) / 111.195
                        arrivals_minus = model.get_travel_times(
                            source_depth_in_km=depth,
                            distance_in_degree=new_distance_minus_deg,
                            phase_list=["P"]
                        )
                        if not arrivals_minus:
                            continue
                        t_minus = arrivals_minus[0].time
                    else:
                        continue

                    # 计算慢度（dt/dr）
                    dt_dr = (t_plus - t_minus) / (2 * delta_r)
                    theoretical_slowness = dt_dr

                    # 计算方位角误差（考虑360°周期性）
                    angle_diff = abs(azimuth - observed_azimuth)
                    angle_diff = min(angle_diff, 360 - angle_diff)
                    angle_error = angle_diff ** 2

                    # 计算慢度误差
                    slowness_error = (theoretical_slowness - observed_slowness) ** 2

                    # 总误差（可调整权重）
                    total_error = slowness_error + angle_error

                    # 更新最优解
                    if total_error < min_error:
                        min_error = total_error
                        best_location = (lat, lon, depth)
                except Exception as e:
                    print(f"在计算 ({lat}, {lon}, {depth}) 时出现错误: {e}")

    return best_location, min_error

# 示例参数设置
if __name__ == "__main__":
    # 输入参数（根据实际数据修改）
    station_lat = 34.0522  # 台站纬度（示例：洛杉矶）
    station_lon = -118.2437  # 台站经度
    min_lat, max_lat, d_lat = 33.0, 35.0, 0.1  # 网格纬度范围及步长（度）
    min_lon, max_lon, d_lon = -119.0, -117.0, 0.1  # 网格经度范围及步长（度）
    min_depth, max_depth, d_depth = 0.0, 100.0, 100.0  # 深度范围及步长（公里）
    observed_slowness = 0.3  # 观测慢度（秒/公里）
    observed_azimuth = 120.0  # 观测方位角（度）

    # 运行定位
    best_location, error = find_source_location(
        station_lat, station_lon,
        min_lat, max_lat, d_lat,
        min_lon, max_lon, d_lon,
        min_depth, max_depth, d_depth,
        observed_slowness, observed_azimuth
    )

    if best_location:
        print("最优震源位置：")
        print(f"纬度：{best_location[0]:.2f}°, 经度：{best_location[1]:.2f}°, 深度：{best_location[2]:.1f} km")
        print(f"最小误差：{error:.4f}")
    else:
        print("未找到合适的震源位置。")
    

最优震源位置：
纬度：33.50°, 经度：-117.10°, 深度：0.0 km
最小误差：0.0968


In [2]:
import numpy as np
from geopy.distance import geodesic
from obspy.taup import TauPyModel

# 输入参数
lat0, lon0 = 30.0, 120.0  # 台阵中心
theta = 45.0               # 方位角（度）
d_min, d_max, delta_d = 0, 500, 1  # 搜索距离参数
s_measured = 0.1           # 实测慢度（s/km）

# 步骤 1：生成候选点
points = []
for d in np.arange(d_min, d_max + delta_d, delta_d):
    # 使用geodesic计算沿方位角的点
    point = geodesic(kilometers=d).destination((lat0, lon0), bearing=theta)
    points.append((point.latitude, point.longitude, d))

# 步骤 2：获取深度（伪代码，需ETOPO1接口）
def get_depth(lat, lon):
    # 假设通过ETOPO1查询高程h（米）
    h = 0 #query_etopo1(lat, lon)
    return -h / 1000  # 转换为公里，假设h海洋为负

# 步骤 3：计算理论慢度
model = TauPyModel(model="iasp91")
residuals = []
for lat, lon, d in points:
    depth = get_depth(lat, lon)
    arrivals = model.get_ray_paths(
        source_depth_in_km=depth,
        distance_in_degree=geodesic((lat0, lon0), (lat, lon)).kilometers / 111.195,
        phase_list=["P"]
    )
    if arrivals:
        p = arrivals[0].ray_param  # 单位：s/°
        s_theory = p / 111.195     # 转换为s/km
        residual = abs(s_theory - s_measured)
        residuals.append(residual)
    else:
        residuals.append(np.inf)

# 步骤 4：选择最优解
best_idx = np.argmin(residuals)
best_lat, best_lon, best_d = points[best_idx]
print(f"最佳震源位置：纬度 {best_lat}, 经度 {best_lon}, 距离 {best_d} km")

最佳震源位置：纬度 33.13424513695119, 经度 123.78783803843956, 距离 500 km
